# Laboratorium 4 - Recurrent Neural Network cz. II

# Zadanie 1 - Wprowadzenie
Stwórz model Simple RNN, który będzie przewidywał ceny mieszkań na podstawie danych z [Boston Housing](https://keras.io/api/datasets/boston_housing/). Większość problemów i implementacji powinna już być znana z poprzednich zajęć więć, prześledzimy to rozwiązanie i poszukamy potencjalnych rozwiązań/ulepszeń.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Dropout,LSTM, GRU
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler


In [ ]:
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
model = Sequential()
model.add(SimpleRNN(units=32, input_shape=(X_train.shape[1], 1)))
model.add(Dense(units=1))
model.summary()

In [ ]:
model.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')
history = model.fit(X_train_scaled[:,:,np.newaxis], y_train, epochs=100, batch_size=32, validation_split=0.2)


In [ ]:
y_pred = model.predict(X_test_scaled[:,:,np.newaxis])
plt.plot(y_test, label='true')
plt.plot(y_pred, label='predicted')
plt.legend()
plt.show()

# Zadanie 1 - Modyfikacje i usprawnienia
Wyniki nie są zadowalające, spróbuj poprawić ten model. Zaproponuj odpowiednie modyfikacje, może LSTM albo GRU zadziałają. Przetestuj i porównaj wyniki. Jakie wnioski się nasuwają?

# Zadanie 2 - Wprowadzenie

**Return_sequence=True** to argument funkcji warstwy rekurencyjnej w sieciach neuronowych, który określa, czy dana warstwa powinna zwrócić sekwencję wyników (ang. sequence) dla każdego kroku czasowego, czy tylko wynik dla ostatniego kroku czasowego.

W przypadku rekurencyjnych sieci neuronowych (RNN) argument ten jest szczególnie ważny, ponieważ sieci RNN są zdolne do przetwarzania sekwencji danych wejściowych. Każdy krok czasowy w sekwencji jest przetwarzany przez sieć RNN, a wynik w każdym kroku jest używany do wyznaczenia wyniku w następnym kroku.

Jeśli **Return_sequence=True**, to warstwa RNN zwróci sekwencję wyników dla każdego kroku czasowego, co oznacza, że wynik na każdym kroku będzie przekazywany do następnego kroku czasowego. To pozwala na dalsze przetwarzanie przez kolejne warstwy sieci lub analizę wyników dla każdego kroku.

Jeśli **Return_sequence=False**, to warstwa RNN zwróci tylko wynik ostatniego kroku czasowego, co oznacza, że wynik na każdym kroku jest zignorowany poza wynikiem ostatniego kroku. To jest przydatne w przypadku, gdy sieć RNN jest używana do generowania jednego wyniku na końcu sekwencji, np. w zadaniach przewidywania lub generowania sekwencji.

Warto zaznaczyć, że niektóre implementacje sieci RNN pozwalają na ustawienie **Return_sequence=False** dla niektórych warstw i **Return_sequence=True** dla innych, co pozwala na bardziej elastyczne modele sieciowe.

In [ ]:
model = Sequential()
model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(SimpleRNN(units=32, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.summary()

## TO DO: Wybierz optymalizator(zastanów sie nad parametrami jakie można mu przypisać oraz dobierz funkcje straty!)
model.compile(optimizer= ??? , loss='')

In [ ]:
history = model.fit(X_train_scaled[:,:,np.newaxis], y_train, epochs=200, batch_size=16, validation_split=0.2)

In [ ]:
y_pred = model.predict(X_test_scaled[:,:,np.newaxis])
plt.plot(y_test, label='true')
plt.plot(y_pred, label='predicted')
plt.legend()
plt.show()

# Zadanie 2 - Modyfikacje i usprawnienia
Dokonaj modyfikacji, zaproponuj rozwiązania oparte o GRU i LSTM. Jakie wnioski płyną z twojej analizy?

# Zadanie 3 Przetestuj [BLSTM](https://analyticsindiamag.com/complete-guide-to-bidirectional-lstm-with-python-codes/).

In [ ]:
from tensorflow.keras.layers import Bidirectional

model = Sequential()
model.add(Bidirectional(LSTM(units=64, input_shape=(X_train.shape[1], 1), return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=32, return_sequences=False)))
model.add(Dense(units=1))
model.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')


In [ ]:
history = model.fit(X_train_scaled[:,:,np.newaxis], y_train, epochs=100, batch_size=16, validation_split=0.2)

In [ ]:
y_pred = model.predict(X_test_scaled[:,:,np.newaxis])
plt.plot(y_test, label='true')
plt.plot(y_pred, label='predicted')
plt.legend()
plt.show()

# Zadanie 4 - LSTM, GRU, SimpleRNN w zadaniach z tekstem (zbiór Shakespearea)

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# Wczytanie danych tekstowych
with open(path_to_file, 'r') as f:
    text = f.read().lower()
print('Długość tekstu:', len(text))

# Tworzenie słownika znaków
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Przygotowanie sekwencji danych wejściowych i wyjściowych
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i:i + maxlen])
    next_chars.append(text[i + maxlen])
print('Liczba sekwencji:', len(sentences))

# Przygotowanie danych wejściowych i wyjściowych
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# Definicja modelu sieci rekurencyjnej
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(256, input_shape=(maxlen, len(chars)), return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(chars), activation='softmax')
])

# Kompilacja modelu
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# Nauczanie modelu
model.fit(x, y, batch_size=128, epochs=50)

# Generowanie tekstu
start_index = np.random.randint(0, len(text) - maxlen - 1)
generated_text = text[start_index:start_index + maxlen]
print('--- Początek generowanego tekstu: "' + generated_text + '"')

for i in range(400):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(generated_text):
        x_pred[0, t, char_indices[char]] = 1
    preds = model.predict(x_pred, verbose=0)[0]
    next_index = np.random.choice(len(preds), p=preds)
    next_char = indices_char[next_index]
    generated_text += next_char
    generated_text = generated_text[1:]
    
print('--- Wygenerowany tekst: "' + generated_text + '"')


# Zadanie 5 - LSTM, GRU, SimpleRNN w zadaniach z tekstem (IMDB, 20groupsnew)
Korzystajac z kodu z zajęć poprzednich oraz wiedzy zdobytej na dzisiejszych zaimplementuj lepsze rozwiązanie LSTM,GRU,SimpleRNN z wykorzystaniem return_sequences dla zbioru IMDB. Wykorzystaj przykłady kodu z poprzednich zajęć, bądź przesłanych przez prowadzacego. Może BLSTM?

# Zadanie 6 - sieć seq2seq

Z racji że nie mamy wystarczająco czasu na omówienie sieci seq2seq przedstawiam wam przykłady dot. wykorzystania sieci seq2seq w tłumaczeniu tekstów:

- [Przykład 1 czesc I](https://blog.paperspace.com/introduction-to-seq2seq-models/)
- [Przykład 1 czesc II](https://blog.paperspace.com/implement-seq2seq-for-text-summarization-keras/)
- [Przykład 2](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html)
- [Przykład 3](https://machinelearningmastery.com/define-encoder-decoder-sequence-sequence-model-neural-machine-translation-keras/)
- [Przykład 4](https://stackabuse.com/python-for-nlp-neural-machine-translation-with-seq2seq-in-keras/)
- [Przykład 5](https://bgg.medium.com/seq2seq-pay-attention-to-self-attention-part-1-d332e85e9aad)



# Zadanie 7 Kompilatory i funkcje straty

Zadanie polega na porównaniu osiąganych wyników trzech różnych modeli (SimpleRNN, LSTM, GRU) dla zbioru danych Housing, wykorzystując różne optymalizatory i funkcje straty.

* Dla modelu 1 (SimpleRNN):
Optymalizator: Adam
Funkcja straty: Mean Squared Error

* Dla modelu 2 (LSTM):
Optymalizator: RMSprop
Funkcja straty: Mean Absolute Error
* Dla modelu 3 (GRU):
Optymalizator: Adagrad
Funkcja straty: Mean Squared Logarithmic Error

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Dropout,LSTM, GRU
from keras.optimizers import Adam, RMSprop, Adagrad
from sklearn.preprocessing import StandardScaler

(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


model_1 = Sequential()
model_1.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], 1), return_sequences=True))
model_1.add(SimpleRNN(units=32, return_sequences=False))
model_1.add(Dropout(0.2))
model_1.add(Dense(units=1))
model_1.summary()

model_2 = Sequential()
model_2.add(LSTM(units=64, input_shape=(X_train.shape[1], 1), return_sequences=True))
model_2.add(LSTM(units=32, return_sequences=False))
model_2.add(Dropout(0.2))
model_2.add(Dense(units=1))
model_2.summary()

model_3 = Sequential()
model_3.add(GRU(units=64, input_shape=(X_train.shape[1], 1), return_sequences=True))
model_3.add(GRU(units=32, return_sequences=False))
model_3.add(Dropout(0.2))
model_3.add(Dense(units=1))
model_3.summary()

In [ ]:
# Model 1
model_1.compile(optimizer=Adam(), loss='mse')
model_1.fit(X_train_scaled[:,:,np.newaxis], y_train, batch_size=32, epochs=50)

In [ ]:
# Model 2
model_2.compile(optimizer=RMSprop(), loss='mae')
model_2.fit(X_train_scaled[:,:,np.newaxis], y_train, batch_size=32, epochs=50)

In [ ]:
# Model 3
model_3.compile(optimizer=Adagrad(), loss='mean_squared_logarithmic_error')
model_3.fit(X_train_scaled[:,:,np.newaxis], y_train, batch_size=32, epochs=50)

In [ ]:
# Predictions
y_pred_1 = model_1.predict(X_test_scaled[:,:,np.newaxis])
y_pred_2 = model_2.predict(X_test_scaled[:,:,np.newaxis])
y_pred_3 = model_3.predict(X_test_scaled[:,:,np.newaxis])

In [ ]:
# Plot predictions vs true values
plt.plot(y_test, label='true')
plt.plot(y_pred_1, label='SimpleRNN')
plt.plot(y_pred_2, label='LSTM')
plt.plot(y_pred_3, label='GRU')
plt.legend()
plt.show()

# Zadanie 8 - Modyfikacje 
Dokonaj modyfikacji. 
Wykorzystujac [funkcje straty](https://keras.io/api/losses/) z biblioteki KERAS oraz dostępne tam [optymalizatory](https://keras.io/api/optimizers/) dokonaj porównania:

- 3 sieci LSTM
- 3 sieci SimpleRNN
- 3 sieci GRU

dla zbioru [Boston Housing](https://keras.io/api/datasets/boston_housing/)

Najistotniejsze w tym zadaniu jest wykorzystanie 3różnych optymalizatorów i funkcji straty dla danej architektury sieci neuronowej i zestawienie wyników. Czy jest zauważalny wpływ na osiągnięte wyniki? Wykorzystaj optymalizatory oraz funkcje straty w sposob **logiczny**.